##  Importer les bibliothèques 

In [1]:
import mlflow
import mlflow.sklearn
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


In [18]:
# Configurer le tracking URI de MLflow (remplacez par votre chemin)
mlflow.set_tracking_uri("/Users/killian/Desktop/Titanic/Notebook/mlruns")


## Créer de nouvelles données

In [23]:
import pandas as pd

# Charger l'ancien ensemble de données
file_path = '../Data/train.csv'  
train_df = pd.read_csv(file_path)

# Afficher les premières lignes pour vérifier
print("Anciennes données:")
print(train_df.head())

# Créer un échantillon des données existantes
new_data = train_df.sample(frac=0.1, random_state=42) 

# Supprimer la colonne 'Survived' 
new_data = new_data.drop(columns=['Survived'])


print("Nouvelles données:")
print(new_data.head())

# Enregistrer les nouvelles données dans un fichier CSV
new_data_path = '../Data/new_data.csv' 
new_data.to_csv(new_data_path, index=False)

print(f"Les nouvelles données ont été enregistrées dans {new_data_path}")

Anciennes données:
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500 

##  Charger les nouvelles données

In [20]:
# Fonction pour charger les données
def load_data(file_path):
    return pd.read_csv(file_path)

# Charger les nouvelles données
new_data_path = '../Data/new_data.csv'  # Remplacez par le chemin réel de vos nouvelles données
new_data = load_data(new_data_path)

# Afficher les premières lignes des nouvelles données pour vérifier
new_data.head()


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,710,3,"Moubarek, Master. Halim Gonios (""William George"")",male,NaN,1,1,2661,15.2458,NaN,C
1,440,2,"Kvillner, Mr. Johan Henrik Johannesson",male,31.0,0,0,C.A. 18723,10.5000,NaN,S
2,841,3,"Alhomaki, Mr. Ilmari Rudolf",male,20.0,0,0,SOTON/O2 3101287,7.9250,NaN,S
3,721,2,"Harper, Miss. Annie Jessie ""Nina""",female,6.0,0,1,248727,33.0000,NaN,S
4,40,3,"Nicola-Yarred, Miss. Jamila",female,14.0,1,0,2651,11.2417,NaN,C


## Obtenir le run ID

In [25]:


# Initialiser le client MLflow
client = MlflowClient()

# Obtenir l'ID de l'expérience (par exemple, l'expérience par défaut)
experiment_name = "Titanic"  # Remplacez par le nom de votre expérience si nécessaire
experiment = client.get_experiment_by_name(experiment_name)
experiment_id = experiment.experiment_id

# Lister tous les runs dans l'expérience
runs = client.search_runs([experiment_id])

# Afficher les Run IDs
for run in runs:
    print("Run ID:", run.info.run_id)



Run ID: 9137012ac2004efb8f86800195d51a9c
Run ID: 0c1f52b37717437f9127b0166fcc0e80
Run ID: a7ea4a26a78245d49f7c289ee7da3f47
Run ID: 862fef78e410483bb79eae1ffe1bba9b
Run ID: 3dca89f67ac04c9c97c856c304a7c0d5
Run ID: b03e8d8721534b7489a0cdf04d1cf23f
Run ID: f5ecc3ecb2e2407688c044dab9cec6d3
Run ID: 5d789c02ac80498c972cd5ce6024bea9
Run ID: bab8007b4e12403cba1ebf29059a9b7d


##  Charger le modèle depuis MLflow

In [22]:
# Spécifiez le Run ID du modèle MLflow que vous voulez charger
run_id =run.info.run_id
model_uri = f"runs:/{mlflow.active_run().info.run_id}/random_forest_model"

# Charger le modèle depuis MLflow
loaded_model = mlflow.sklearn.load_model(model_uri)

# Vérifiez que le modèle est bien chargé
loaded_model


AttributeError: 'NoneType' object has no attribute 'info'

## Préparer les nouvelles données

In [8]:
# Vérifier si la colonne 'Survived' existe
if 'Survived' in new_data.columns:
    print("La colonne 'Survived' a été trouvée, on la supprime.")
    X_new = new_data.drop(columns=['Survived'])  # Exemple de prétraitement
else:
    print("La colonne 'Survived' n'a pas été trouvée, on continue sans la supprimer.")
    X_new = new_data.copy()  # Assurez-vous que 'PassengerId' existe pour la prédiction

# Afficher les premières lignes des nouvelles données après prétraitement
print("Premières lignes des nouvelles données après prétraitement:")
print(X_new.head())


La colonne 'Survived' n'a pas été trouvée, on continue sans la supprimer.
Premières lignes des nouvelles données après prétraitement:
   PassengerId  Pclass                                               Name  \
0          710       3  Moubarek, Master. Halim Gonios ("William George")   
1          440       2             Kvillner, Mr. Johan Henrik Johannesson   
2          841       3                        Alhomaki, Mr. Ilmari Rudolf   
3          721       2                  Harper, Miss. Annie Jessie "Nina"   
4           40       3                        Nicola-Yarred, Miss. Jamila   

      Sex   Age  SibSp  Parch            Ticket     Fare Cabin Embarked  
0    male   NaN      1      1              2661  15.2458   NaN        C  
1    male  31.0      0      0        C.A. 18723  10.5000   NaN        S  
2    male  20.0      0      0  SOTON/O2 3101287   7.9250   NaN        S  
3  female   6.0      0      1            248727  33.0000   NaN        S  
4  female  14.0      1      0    

## Appliquer le modèle pour faire des prédictions

In [24]:
# Appliquer le modèle pour faire des prédictions
predictions = loaded_model.predict(X_new)

# Générer le fichier submission.csv
submission_pred = pd.DataFrame({'PassengerId': new_data['PassengerId'], 'Survived': predictions})
submission_pred.to_csv('/submission_pred.csv', index=False)

print("Les prédictions ont été enregistrées dans submission_pred.csv")


NameError: name 'loaded_model' is not defined